# Notebook to generate dataset of MPC I/O examples

In [ ]:
from mc_utils_LT import *
from scipy.linalg import block_diag
from scipy.io import savemat
# casadi/acados model
import casadi as ca
from casadi import SX, sqrt, atan, tan, sin, cos, tanh, atan2, fabs, vertcat, if_else, sign
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver, AcadosSim, AcadosSimSolver
# suppress warnings acados
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='acados_template')
# delete c_generated_code directory if it exists
import shutil
import os
if os.path.exists('c_generated_code'): shutil.rmtree('c_generated_code')